# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from ipywidgets import interact, interact_manual

In [3]:
from data_processing import fill_demean_scale

# Read in `train` and `test` data

In [4]:
train_df = pd.read_csv("./train.csv", index_col=0)

In [5]:
test_df = pd.read_csv("./test.csv", index_col=0)

# Explore dataset

## dtypes

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.dtypes

We want to predict whether a passenger survived the Titanic or not. The target or label in this problem is therefore `Survived`.

This is a categorical variable (1 or 0) and therefore we are dealing with a (binary) classification problem.

On what types of information are our predictions to be based?

## schema

`Pclass`: 1st, 2nd, or 3rd class ticket (int, categorical variable)

`Name`: full name, string

`Sex`: takes values `male` or `female`

`Age`: age in years, float

`SibSp`: the number of siblings or spouses that passenger had onboard, int

`Parch`: the number of parents and children that passenger had onboard, int

`Ticket`: the ticket number (non-standardised, a few different formats, str

`Fare`: the cost of the tickets (in standardised currency???), float

`Cabin`: the cabin code of each passenger, string

`Embarked`: where they got on the boat (Cherbourg, Queenstown, Southampton), str (categorical)

In [ ]:
@interact
def show_me_unique_values(column=sorted(list(train_df.columns))):
    return train_df[column].unique()

## `Age`

The mean passenger age was 29.7, with 44 0 to 5 year-olds.

In [ ]:
train_df.Age.mean()

In [ ]:
train_df[train_df.Age <= 5.0]['Name'].count()

In [ ]:
train_df.Age.plot(kind='hist', bins=20)

## `Pclass`

There were actually fewer second class tickets than first class tickets. Who woulda thought...

In [ ]:
assert (train_df.Pclass.value_counts() / train_df.Pclass.size).sum() == 1.0

train_df.Pclass.value_counts() / train_df.Pclass.size

In [ ]:
train_df.Pclass.plot(kind='hist')

## `SibSp`

Most of the passengers travelled with neither siblings nor spouses.

This may mean that most travelled alone, or that they travelled with a parent or child.

In [ ]:
train_df.SibSp.value_counts()

In [ ]:
train_df.SibSp.plot(kind='hist')

## `Parch`

Most indeed travelled alone.

In [ ]:
train_df.Parch.value_counts()

In [ ]:
train_df.Parch.plot(kind='hist')

## `Fare`

In [ ]:
train_df.plot(
    kind='scatter', y='Fare', x='Age', c='Pclass', cmap='Set1',legend=True
)

At first glance it seems that `Fare` doesn't tell us much about `Age`.

## `Cabin`

In [ ]:
# train_df['Cabin_prefix'] = train_df.Cabin.apply(
#     lambda x: str(x)[0] if str(x) != 'nan' else 'nan'
# )

In [ ]:
train_df.Cabin_prefix.unique()

There are seven different `Cabin_prefix`es from `A` through `F` plus `T`. What did they mean?

There are also some dirty data entries in `Cabin` such as "F G63".

## `Embarked`

In [ ]:
embarked_mapping = {'Q':1, 'S':2, 'C':3}

train_df['Embarked'] = train_df['Embarked'].map(embarked_mapping)

In [ ]:
train_df.plot(kind='scatter', x='Age', y='Fare', c='Embarked', cmap='Set1')

## `test_df`

In [ ]:
test_df.describe()

# Process `train_df` data

## `Sex`

In [7]:
sex_mapping = {'male':1,'female':0}
train_df.Sex = train_df.Sex.map(sex_mapping)

## `Age`, `Fare`, `Parch`, `SibSp`

In [8]:
for column in ['Age', 'Fare', 'Parch', 'SibSp']:
    train_df[column] = fill_demean_scale(df=train_df, column=column)

## Drop `Embarked` nulls

In [9]:
train_df = train_df[~train_df.Embarked.isnull()]
embarked_mapping = {'Q':1, 'S':2, 'C':3}

train_df['Embarked'] = train_df['Embarked'].map(embarked_mapping)

## Drop `Cabin`

In [10]:
train_df = train_df.drop(columns='Cabin', axis=1)

## Checks

In [12]:
train_df.columns

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Embarked'],
      dtype='object')

In [13]:
train_df.isnull().sum().sum()

0

# Process `test_df` data

## `Sex`

In [21]:
sex_mapping = {'male':1,'female':0}
test_df.Sex = test_df.Sex.map(sex_mapping)

## `Age`, `Fare`, `Parch`, `SibSp`

In [14]:
for column in ['Age', 'Fare', 'Parch', 'SibSp']:
    test_df[column] = fill_demean_scale(df=test_df, column=column)

## Drop `Embarked` nulls

In [15]:
test_df = test_df[~test_df.Embarked.isnull()]

embarked_mapping = {'Q':1, 'S':2, 'C':3}
test_df['Embarked'] = test_df['Embarked'].map(embarked_mapping)

## Drop `Cabin`

In [16]:
test_df = test_df.drop(columns='Cabin', axis=1)

In [17]:
test_df.columns

Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Embarked'],
      dtype='object')

## Checks

In [22]:
test_df.dtypes

Pclass        int64
Name         object
Sex           int64
Age         float64
SibSp       float64
Parch       float64
Ticket       object
Fare        float64
Embarked      int64
dtype: object

In [23]:
test_df.isnull().sum()

Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Embarked    0
dtype: int64

In [24]:
assert len(train_df.columns) - 1 == len(test_df.columns)

In [25]:
train_df.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000
mean,0.382452,2.311586,0.649044,-0.000574,0.000147,0.000143,-0.000210,2.102362
std,0.486260,0.834700,0.477538,0.162960,0.137963,0.134460,0.097003,0.515181
min,0.000000,1.000000,0.000000,-0.367921,-0.065376,-0.063599,-0.062858,1.000000
25%,0.000000,2.000000,0.000000,-0.096747,-0.065376,-0.063599,-0.047447,2.000000
50%,0.000000,3.000000,1.000000,0.000000,-0.065376,-0.063599,-0.034646,2.000000
75%,1.000000,3.000000,1.000000,0.066611,0.059624,-0.063599,-0.002350,2.000000
max,1.000000,3.000000,1.000000,0.632079,0.934624,0.936401,0.937142,3.000000


In [26]:
test_df.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,418.000000,418.000000,4.180000e+02,4.180000e+02,4.180000e+02,4.180000e+02,418.000000
mean,2.265550,0.636364,3.187243e-18,-3.187243e-18,-2.921640e-18,-1.062414e-18,2.133971
std,0.841838,0.481622,1.666166e-01,1.120949e-01,1.090477e-01,1.089934e-01,0.580452
min,1.000000,0.000000,-3.969747e-01,-5.592105e-02,-4.359383e-02,-6.953964e-02,1.000000
25%,1.000000,0.000000,-9.590651e-02,-5.592105e-02,-4.359383e-02,-5.412807e-02,2.000000
50%,3.000000,1.000000,0.000000e+00,-5.592105e-02,-4.359383e-02,-4.132692e-02,2.000000
75%,3.000000,1.000000,7.223275e-02,6.907895e-02,-4.359383e-02,-8.055735e-03,2.000000
max,3.000000,1.000000,6.030253e-01,9.440789e-01,9.564062e-01,9.304604e-01,3.000000


# `X_train, y_train, X_test`

In [28]:
cols_to_drop = ["Name", "Ticket", "Survived"]

features = [col for col in train_df.columns if col not in cols_to_drop]

X_train, y_train = train_df[features], train_df.Survived

In [29]:
X_test = test_df[features]

# Model

In [30]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()

In [31]:
clf.fit(X_train, y_train)

LogisticRegression()

In [33]:
predictions = clf.predict(X_test)

In [35]:
test_df['Survived'] = predictions

In [36]:
test_df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Predicted
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",1,0.055749,-0.055921,-0.043594,330911,-0.054258,1,0
893,3,"Wilkes, Mrs. James (Ellen Needs)",0,0.220591,0.069079,-0.043594,363272,-0.055877,2,0
894,2,"Myles, Mr. Thomas Francis",1,0.418402,-0.055921,-0.043594,240276,-0.050631,1,0
895,3,"Wirz, Mr. Albert",1,-0.043157,-0.055921,-0.043594,315154,-0.052632,2,0
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,-0.109094,0.069079,0.067517,3101298,-0.045556,2,1


In [38]:
test_df['Survived'].value_counts()

0    263
1    155
Name: Predicted, dtype: int64

In [39]:
test_df["Survived"].to_csv("predicted_values.csv")